# DistilBERT model (train: Waseem & Hovy, test: Waseem & Hovy)
This notebook trains the DistilBERT model and exports a set of predictions for a test dataset.

First we need to install the required packages.

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 34.1 MB/s 
     |████████████████████████████████| 101 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 34.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 64.7 MB/s 
     |████████████████████████████████| 140 kB 61.9 MB/s 
     |████████████████████████████████| 1.1 MB 43.0 MB/s 
     |████████████████████████████████| 127 kB 42.3 MB/s 
     |████████████████████████████████| 271 kB 39.3 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 40.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizer
import numpy as np
from datasets import load_metric, load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pickle
import torch
from google.colab import drive
import preprocessor as p
import html
from torch.utils.data import DataLoader
import torch.nn.functional as nn


# Mount drive for loading the datasets
drive.mount('/content/drive')
import sys
sys.path.insert(0, '/content/drive/MyDrive/Colab Notebooks/')

FILENAME = "drive/MyDrive/Colab Notebooks/data/twitter_data.pkl"
NUM_LABELS = 2
BATCH_SIZE = 256
NUM_EPOCHS = 5

Mounted at /content/drive


## Split and tokenize the datasets

In [6]:
class HateDataset(torch.utils.data.Dataset):
    """Dataset class used for combining the data encodings and labels."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def preprocess(data):
    """Preprocesses the data by tokenizing URLs and MENTIONS since these do not
    contain any valuable information."""
    p.set_options(p.OPT.URL, p.OPT.MENTION)

    return list(map(lambda text: p.tokenize(html.unescape(text)), data))

def convert_waseem_data(data):
    """Converts the Waseem and Hovy dataset by creating binary labels"""
    X = []
    y = []
    for i in range(len(data)):
        X.append(data[i]['text'])
        label = data[i]['label']
        if label in ['racism', 'sexism']:
          label = 1
        else:
          label = 0
        y.append(label)

    return X, y

In [7]:
data = pickle.load(open(FILENAME, 'rb'))
X, y = convert_waseem_data(data)
X = preprocess(X)

# Split dataset into train, test, and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.2)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize all datasets
train_encodings = tokenizer(X_train, truncation=True, padding=True)
val_encodings = tokenizer(X_val, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

# Combine the encodings with the labels to Torch datasets
train_dataset = HateDataset(train_encodings, y_train)
val_dataset = HateDataset(val_encodings, y_val)
test_dataset = HateDataset(test_encodings, y_test)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

## Load accuracy metric for the model's evaluation

In [8]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Setup DistilBERT model

In [9]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=NUM_LABELS)
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=BATCH_SIZE, num_train_epochs=NUM_EPOCHS)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset ,
    compute_metrics=compute_metrics
)

## Training

In [10]:
trainer.train()

## Export

In [11]:
path = F"drive/MyDrive/Colab Notebooks/output/distilbert-waseem-hovy.pth" 
torch.save(model, path)

## Load existing model

In [12]:
old_model = torch.load(F"drive/MyDrive/Colab Notebooks/output/distilbert-waseem-hovy.pth")
old_model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

## Model calibration
We use temperature scaling to calibrate the model on the validation set by finding the optimal T value.

In [20]:
import sys
sys.path.append("drive/MyDrive/Colab Notebooks")
from temperature_scaling import ModelWithTemperature


In [22]:
calibrated_model = ModelWithTemperature(old_model)
val_loader = DataLoader(val_dataset)

# Find optimal T value to calibrate the model
calibrated_model.set_temperature(val_loader)


Before temperature - NLL: 1.148, ECE: 0.408
Optimal temperature: 1.797
After temperature - NLL: 0.843, ECE: 0.325


ModelWithTemperature(
  (model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (

## Run calibrated model on the test dataset
The predicted output logits of the test dataset are converted to class probabilities using Softmax.

In [33]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

logits = []
labels = []

test_loader = DataLoader(test_dataset)

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label = batch['labels'].to(device)
        outputs = calibrated_model(input_ids, attention_mask=attention_mask)
        logits.append(outputs.tolist()[0])
        labels.append(label)

probabilities = nn.softmax(torch.tensor(logits), dim=1)
print(probabilities.tolist()[:10])

[[0.957335889339447, 0.04266403988003731], [0.1251574009656906, 0.8748425245285034], [0.10404887050390244, 0.8959511518478394], [0.8604177832603455, 0.13958224654197693], [0.10650099813938141, 0.893498957157135], [0.14507947862148285, 0.854920506477356], [0.9497029185295105, 0.0502970814704895], [0.9488005042076111, 0.05119951814413071], [0.9570857882499695, 0.042914219200611115], [0.9382002353668213, 0.06179974600672722]]


In [35]:
predictions = torch.argmax(probabilities, dim=-1)
accuracy = metric.compute(predictions=predictions, references=labels)
print("Test dataset: ", accuracy)

Test dataset:  {'accuracy': 0.8520820385332505}


## Export predictions

In [39]:
import numpy as np

def to_predictions(predictions, labels):
    predicted_classes = np.argmax(predictions, 1)
    return list(map(lambda predicted_values, predicted_class, actual_class:
                    {'predicted_class': class_name(predicted_class),
                     'actual_class': class_name(actual_class),
                     'predicted_value': predicted_values[predicted_class],
                     'text': None},
                    predictions, predicted_classes, labels))
def class_name(index):
    if index == 0:
        return "None"
    elif index == 1:
        return "Hate"

predictions_info = to_predictions(probabilities, labels)
pickle.dump(predictions_info, open("drive/MyDrive/Colab Notebooks/output/distilbert-waseem-waseem.p", "wb"))